## Get names and definitions for each image class used in natural image stimuli

In [93]:
import scipy.io
import numpy as np

mat = scipy.io.loadmat('../other data/meta.mat')
synsets = np.array(mat["synsets"])

imagenet_wnid_list = [row[0][1][0] for row in synsets]

print(f'imagenet_list_len: {len(imagenet_wnid_list)}')

imagenet_list_len: 1905


In [95]:
train_wnid_list = []
with open("../other data/stimulus_perceptionNaturalImageTraining.tsv") as f:
    f = list(f)
    train_wnid_list = [r.split('\t')[0][:9] for r in f]

test_wnid_list = []
with open("../other data/stimulus_perceptionNaturalImageTest.tsv") as f:
    f = list(f)
    test_wnid_list = [r.split('\t')[0][:9] for r in f]

fmri_wnid_list = train_wnid_list + test_wnid_list

print(f'fmri_list_len: {len(fmri_wnid_list)}')

fmri_list_len: 1250


In [96]:
imagenet_wnid_set = set(imagenet_wnid_list)
fmri_wnid_set = set(fmri_wnid_list)

In [102]:
print(f'imagenet_set: {len(imagenet_wnid_set)}')
print(f'fmri_set: {len(fmri_wnid_set)}')

in_fmri_not_imagenet = fmri_wnid_set - fmri_wnid_set.intersection(imagenet_wnid_set)  # A n (B^)
print(f'in_fmri_not_imagenet: {len(in_fmri_not_imagenet)}')

imagenet_set: 1905
fmri_set: 200
in_fmri_not_imagenet: 89


In [121]:
fmri_unique_wnid_list = list(fmri_wnid_set)
fmri_unique_wnid_list.sort()

base_url = 'http://wordnet-rdf.princeton.edu/json/pwn30/'

wnid_dict = {}
for wnid in fmri_unique_wnid_list:
    wnid_dict[wnid] = {'url' : f'{base_url}{wnid[1:]}-{wnid[0]}'}

In [132]:
import requests
from random import randint

random_wnid = fmri_unique_wnid_list[randint(0,len(fmri_unique_wnid_list))]

sample_request_json = requests.get(wnid_dict[random_wnid]['url']).json()[0]
print(sample_request_json)

{'definition': 'a circular rounded projection or protuberance', 'examples': [], 'lemmas': [{'lemma': 'boss', 'language': 'en', 'forms': [], 'sense_key': 'boss%1:06:00::', 'subcats': [], 'importance': 1}, {'lemma': 'knob', 'language': 'en', 'forms': [], 'sense_key': 'knob%1:06:01::', 'subcats': [], 'importance': 2}], 'id': '03631406-n', 'ili': 'i55436', 'pos': 'n', 'subject': 'noun.artifact', 'relations': [{'src_word': 'boss', 'trg_word': 'boss', 'rel_type': 'derivation', 'target': '01534053-v'}, {'src_word': None, 'trg_word': None, 'rel_type': 'hypernym', 'target': '04015323-n'}, {'src_word': None, 'trg_word': None, 'rel_type': 'hyponym', 'target': '03631709-n'}, {'src_word': None, 'trg_word': None, 'rel_type': 'hyponym', 'target': '03810914-n'}], 'old_keys': {'pwn16': ['02895469-n', '03433357-n'], 'pwn20': ['03491305-n'], 'pwn30': ['03626115-n'], 'pwn15': ['02687864-n'], 'pwn21': ['03586091-n'], 'pwn171': ['03158299-n', '03785363-n'], 'pwn17': ['03093984-n', '03710719-n']}, 'gloss': N

In [ ]:
import requests
import time

print(f'total: {len(fmri_unique_wnid_list)}')

count = 0
for wnid in fmri_unique_wnid_list:

    req_json = requests.get(wnid_dict[wnid]['url']).json()[0]
    names = [lemma['lemma'] for lemma in req_json['lemmas']]
    definition = req_json['definition']

    print(f'{count}\t{names}\n\t{definition}')

    wnid_dict[wnid]['names'] = names
    wnid_dict[wnid]['definition'] = definition

    count += 1
    time.sleep(2)

In [137]:
import json

with open('../other data/fMRI_wnids.json', 'w') as file:
    json.dump(wnid_dict, file, sort_keys=True, indent=4)